In [1]:
import pandas as pd
import numpy as np
import theano.tensor as T
import theano
import theano.gradient
import pyipopt

In [2]:
inputfile = './data_new_volume.csv'
df = pd.read_csv(inputfile)
df['const'] = 1

# df = df.sample(2000)

df['pgmidgrade_km_adj'].fillna(value=10000, inplace=True)

df = df[df['treattype'] < 3]
for elem in df['treattype'].unique():
    df['treat' + str(elem)] = df['treattype'] == elem

for elem in df['choice'].unique():
    df['choice' + str(elem)] = df['choice'] == elem

df['treat1_topusage'] = df['treat1']*df['dv_usageveh_p75p100']
df['treat2_topusage'] = df['treat2']*df['dv_usageveh_p75p100']

df['treat1_college'] = df['treat1']*df['dv_somecollege']
df['treat2_college'] = df['treat2']*df['dv_somecollege']

df.head()

In [3]:
#df = df[ df['dv_carpriceadj_p75p100'] == 1]
#df = df[ df['dv_usageveh_p75p100'] == 0]
df.head()

In [4]:
price_labels = ['pg_km_adj', 'pgmidgrade_km_adj', 'pe_km_adj']
value_labels = ['value_total']

# all
Xexpd_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# usage
#Xexpd_labels = ['choice2', 'choice3', 'dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# car price
#Xexpd_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

Xlelas_labels = ['const', 'treat1', 'treat2', 'dv_usageveh_p75p100', 'treat1_topusage', 'treat2_topusage']
Xlsigma_labels = ['const']
Xlmu_labels = ['const']

In [5]:
floatX = 'float64'

In [6]:
choice = df.loc[:, 'choice'].as_matrix() - 1
price  = df.loc[:, price_labels].as_matrix().astype(floatX)
value  = df.loc[:, value_labels].as_matrix().astype(floatX)
Xexpd  = df.loc[:, Xexpd_labels].as_matrix().astype(floatX)
Xutil  = df.loc[:, Xutil_labels].as_matrix().astype(floatX)

Xlelas = df.loc[:, Xlelas_labels].as_matrix().astype(floatX)
Xlsigma = df.loc[:, Xlsigma_labels].as_matrix().astype(floatX)
Xlmu   = df.loc[:, Xlmu_labels].as_matrix().astype(floatX)

In [7]:
nobs, nchoice = price.shape
nobs, nXexpd = Xexpd.shape
nobs, nXutil = Xutil.shape

nobs, nXlelas = Xlelas.shape
nobs, nXlsigma = Xlsigma.shape
nobs, nXlmu = Xlmu.shape

In [8]:
chosenprice = price[range(nobs), choice].reshape(nobs,1)
dvchoice = (choice == np.arange(nchoice).reshape(nchoice,1)).transpose()
convenience_expend = 50.0
dvconvenience = np.abs(value.squeeze()-convenience_expend) < 1e-3
n_convenience = dvconvenience.sum()
convenience, = np.where(dvconvenience)
inconvenience, = np.where(~dvconvenience)

In [9]:
ndraws = 10
draws = np.random.rand(n_convenience, ndraws)

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(draws)

In [11]:
theta0 = np.zeros(nXlelas + nXlsigma + nXlmu + nXexpd + (nchoice-1)*nXutil)
theta0[0] = 0.1
theta00 = np.hstack([theta0, [0.1]])

In [12]:
def getparams(theta):
    offset = 0
    gammalelas = theta[offset:offset+nXlelas].reshape((nXlelas, 1))
    
    offset += nXlelas
    gammalsigma = theta[offset:offset+nXlsigma].reshape((nXlsigma, 1))
    
    offset += nXlsigma
    gammalmu = theta[offset:offset+nXlmu].reshape((nXlmu, 1))
    
    offset += nXlmu
    betaexpd = theta[offset:offset+nXexpd].reshape((nXexpd, 1))
    
    offset += nXexpd
    betautil = theta[offset:offset+(nchoice-1)*nXutil].reshape((nXutil, nchoice-1))
    
    offset += (nchoice-1)*nXutil
    ltpconve = theta[offset]
    
    return gammalelas, gammalsigma, gammalmu, betaexpd, betautil, ltpconve

theta = T.dvector('theta')
gammalelas, gammalsigma, gammalmu, betaexpd, betautil, ltpconve = getparams(theta)

pconve = T.nnet.sigmoid(ltpconve)

# priceT = theano.shared(price.astype(floatX), name='price')
# pchoiceT = theaeno.shared(pchoice.astype(floatX), name='pchoice')
# XexpdT = theano.shared(Xexpd.astype(floatX), name='Xexpd')
# XutilT = theano.shared(Xutil.astype(floatX), name='Xutil')
# valueT = theano.shared(value.astype(floatX), name='value')
# XlelasT = theano.shared(Xlelas.astype(floatX), name='Xlelas')
# XlsigmaT = theano.shared(Xlsigma.astype(floatX), name='Xlsigma')
# XlmuT = theano.shared(Xlmu.astype(floatX), name='Xlmu')

def logsumexp(x,axis):
    maxx = T.max(x,axis=axis,keepdims=True)
    return maxx.squeeze() + T.log(T.sum(T.exp(x-maxx),axis=axis))

def logsumexp2(x,y):
    m = T.maximum(x,y)
    return m + T.log(T.exp(x-m) + T.exp(y-m))

dvchoicef = dvchoice.astype(floatX)

mu = T.exp(T.dot(Xlmu, gammalmu))
elas = T.exp(T.dot(Xlelas, gammalelas))
lsigma = T.dot(Xlsigma,gammalsigma)
rho = elas - 1

lconvenience_expend = np.log(convenience_expend)

eta = np.log(value) + rho*np.log(chosenprice) - T.dot(Xexpd,betaexpd)
lexpend = np.log(value) - rho*(np.log(price) - np.log(chosenprice))

utilhete = T.concatenate([np.zeros((nobs,1)), T.dot(Xutil, betautil)],axis=1)
utilquant = T.exp(lexpend)/rho

util0 = (utilquant + utilhete)/mu
lprobchoice0 = T.sum(util0*dvchoicef,axis=1) - logsumexp(util0,1)

lprobchoice = lprobchoice0 + np.log(1-pconve)
lpdfeta = eta*eta/(2*T.exp(2*lsigma)) + lsigma + np.log(2*np.pi)/2
nlogl = -lprobchoice.sum() + lpdfeta.sum()

In [13]:
plt.hist(value,20)

In [14]:
# https://pomax.github.io/bezierinfo/legendre-gauss.html
weight = np.array([0.2491470458, 0.2491470458, 0.2334925365, 0.2334925365, 
                   0.2031674267, 0.2031674267, 0.1600783285, 0.1600783285, 
                   0.106939326, 0.106939326, 0.0471753364, 0.0471753364]).reshape(1,-1)
abscissa = np.array([-0.1252334085, 0.1252334085, -0.367831499, 0.367831499, 
                     -0.5873179543, 0.5873179543, -0.7699026742, 0.7699026742, 
                     -0.9041172564, 0.9041172564, -0.9815606342, 0.9815606342]).reshape(1,-1)

# Hermite Gaussian quadrature
# http://keisan.casio.com/exec/system/1329114617
abscissa = np.array([-3.88972489786978000, -3.02063702512089000, -2.27950708050106000, -1.59768263515260000, 
                   -0.94778839124016300, -0.31424037625435900, 0.31424037625435900, 0.94778839124016300, 
                   1.59768263515260000, 2.27950708050106000, 3.02063702512089000, 3.88972489786978000
                  ]).reshape(-1,1,1)

weight = np.array([0.00000026585516844, 0.00008573687043588, 0.00390539058462906, 0.05160798561588390, 
                     0.26049231026416100, 0.57013523626248000, 0.57013523626248000, 0.26049231026416100, 
                     0.05160798561588390, 0.00390539058462906, 0.00008573687043588, 0.00000026585516844
                     ]).reshape(-1,1,1)


### Gauss - Hermite quadrature
$$\int_{-\infty}^{\infty} e^{-x^2}f(x)dx \approx \sum w_i f(x_i)$$

In [15]:
# Phi(a,b)
import scipy.stats
a, b = -0.4, 0.2
x = abscissa*(b-a)/2 + (b+a)/2

truevalue = scipy.stats.norm.cdf(b) - scipy.stats.norm.cdf(a)
approximation = np.sum(scipy.stats.norm.pdf(x)*weight.squeeze())*(b-a)/2
print truevalue, approximation

In [16]:
eta_i = T.exp(lsigma)*abscissa
lexpend_i = lexpend - eta + eta_i
utilconve_i = (T.exp((lexpend_i-lconvenience_expend)/elas)/(1-1/elas) - 1)*convenience_expend
utilb = (utilconve_i + utilhete)/mu
lprobchoice_i = T.sum(utilconve_i*dvchoicef,axis=2) - logsumexp(utilconve_i,2)
ll = lprobchoice_i + np.log(weight)[:,:,0] - np.log(2*np.pi)/2 + T.log(pconve)
ll2 = logsumexp(ll,0)

In [17]:
nlogl_total = nlogl - ll2[convenience].sum()

In [18]:
theano.function([theta], nlogl_total)(theta00)

In [19]:
def buildfunc(theta, nloglf):
    return (theano.function([theta], nloglf),
            theano.function([theta], T.grad(nloglf, theta)),
            theano.function([theta], theano.gradient.hessian(nloglf, theta)))

eval_f, eval_grad, eval_hess = buildfunc(theta, nlogl_total)
# eval_f = theano.function([theta], nlogl_total)
# eval_grad = theano.function([theta], T.grad(nlogl_total, theta))

In [20]:
eval_f(theta00)

In [21]:
pyipopt.set_loglevel(1)
thetahat , _, _, _, _, fval = pyipopt.fmin_unconstrained(
    eval_f,
    theta00,
    fprime=eval_grad,
    fhess=eval_hess,
    )


In [22]:
theano.function([theta], elas)(thetahat)

In [23]:

covhat = np.linalg.inv(eval_hess(thetahat))
sehat = np.sqrt(np.diagonal(covhat))
t = thetahat/sehat

gammalelashat, gammalsigmahat, gammalmuhat, betaexpdhat, betautilhat, bhat = getparams(thetahat)
gammalelasse, gammalsigmase, gammalmuse, betaexpdse, betautilse, bse = getparams(sehat)
gammalelast, gammalsigmat, gammalmut, betaexpdt, betautilt, bt = getparams(t)

formatstr = '%30s%10.3f%10.3f%10.3f'

print '-'*60

print ' \n*** ln(elas) equation'
for i in range(nXlelas):
    print formatstr % (Xlelas_labels[i], gammalelashat[i], gammalelasse[i], gammalelast[i])
print '-'*60

print ' \n*** ln(sigma) equation'
for i in range(nXlsigma):
    print formatstr % (Xlsigma_labels[i], gammalsigmahat[i], gammalsigmase[i], gammalsigmat[i])
print '-'*60

print ' \n*** ln(mu) equation'
for i in range(nXlmu):
    print formatstr % (Xlmu_labels[i], gammalmuhat[i], gammalmuse[i], gammalmut[i])
print '-'*60

print ' \n*** Expenditure equation'
for i in range(nXexpd):
    print formatstr % (Xexpd_labels[i], betaexpdhat[i], betaexpdse[i], betaexpdt[i])
print '-'*60
    
print ' \n*** Discrete choice equation'
for j in range(nchoice-1):
    print '-------- choice', j+1, '------------------------------------------'
    for i in range(nXutil):
        print formatstr % (Xutil_labels[i], betautilhat[i][j], betautilse[i][j], betautilt[i][j])
print '-'*60


In [24]:
theano.function([theta], gammalelas[1,0])(thetahat)

In [25]:
dtreat = gammalelas[1,0] - gammalelas[2,0]
grad_dtreat = T.grad(dtreat, theta)

In [26]:
grad_value = theano.function([theta], grad_dtreat)(thetahat)
se_dtreat = np.sqrt(grad_value.dot(covhat).dot(grad_value))
dtreat_hat = theano.function([theta], dtreat)(thetahat)

In [27]:
dtreat_hat/se_dtreat

In [28]:
nobs

In [29]:
theano.function([theta], pconve)(thetahat)